In [1]:
import os
import numpy as np
import sys
import holoviews as hv

sys.path.append('/Users/annateruel/phd_code/')
from ca2img.minian_dca1 import (preprocessing,  
                                start_cluster,
                                motion_correction,
                                initialize)

from minian.utilities import (open_minian,
                              get_optimal_chk,
                              save_minian)
from minian.motion_correction import apply_transform, estimate_motion
from minian.visualization import visualize_motion

In [7]:
video_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope'
int_dir =  '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope'

In [ ]:
hv.notebook_extension("bokeh", width=100)

# Minian Analysis in dCA1


In [6]:
start_cluster(n_workers=8, memory_limit="6GB")

Client Scheduler: tcp://127.0.0.1:50294 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 16 Memory: 48.00 GB


### Preprocessing

In [8]:
preprocessing(intpath=int_dir, dpath=video_dir, 
                          param_load_videos={"pattern": "^[0-9]+\.avi$", 
                                             "dtype": np.uint8, 
                                             "downsample": dict(frame=1, height=1, width=1), 
                                             "downsample_strategy": "subset"}, 
                          param_denoise={"method": "median", "ksize": 7}, 
                          param_background_removal={"method": "tophat", "wnd": 15})

loading 4 videos in folder /Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope
Number of videos loaded: 3794


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developersffmpeg version 7.0
ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers  built with Apple clang version 15.0.0 (clang-1500.3.9.4)

  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-l

In [ ]:
dirs = ['/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-126/annateruel/social-discrimination-test/AD23-126/customEntValHere/2024_03_12']

for dir in dirs:
    for root, _, _ in os.walk(dir):
        if root.endswith("My_V4_Miniscope"):
            preprocessing(intpath=root, dpath=root, 
                          param_load_videos={"pattern": "^[0-9]+\.avi$", 
                                             "dtype": np.uint8, 
                                             "downsample": dict(frame=1, height=1, width=1), 
                                             "downsample_strategy": "subset"}, 
                          param_denoise={"method": "median", "ksize": 7}, 
                          param_background_removal={"method": "tophat", "wnd": 15})

### Motion correction

In [9]:
param_estimate_motion = {"dim": "frame"}

param_save_minian = {
    "dpath": video_dir,  
    "meta_dict": dict(session=-1, animal=-2),
    "overwrite": True}

minian_ds = open_minian(int_dir)
varr_ref = minian_ds['varr_ref']
chk, _ = get_optimal_chk(varr_ref, dtype=float)
motion_correction(int_dir, varr_ref, param_estimate_motion, param_save_minian, chk)

In [ ]:
root_dir =  '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/'

param_estimate_motion = {"dim": "frame"}

for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'My_V4_Miniscope' in dirnames:
        int_dir = os.path.join(dirpath, 'My_V4_Miniscope')
        minian_ds = open_minian(int_dir)
        varr_ref = minian_ds['varr_ref']
        chk, _ = get_optimal_chk(varr_ref, dtype=float)

        param_save_minian = {
            "dpath": int_dir,  
            "meta_dict": dict(session=-1, animal=-2),
            "overwrite": True}

        %time 
        motion_correction(int_dir, varr_ref, param_estimate_motion, param_save_minian, chk)

### Initialization

In [10]:
int_dir =  '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope'
os.environ["MINIAN_INTERMEDIATE"] = int_dir

param_save_minian = {
    "dpath": int_dir,
    "meta_dict": dict(session=-1, animal=-2),
    "overwrite": True}
param_seeds_init = {
    "wnd_size": 1000,
    "method": "rolling",
    "stp_size": 500,
    "max_wnd": 15,
    "diff_thres": 3}
param_pnr_refine = {"noise_freq": 0.06, "thres": 1}
param_ks_refine = {"sig": 0.05}
param_seeds_merge = {"thres_dist": 10, "thres_corr": 0.8, "noise_freq": 0.06}
param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.06}
param_init_merge = {"thres_corr": 0.8} 

varr_ref = open_minian(int_dir)['varr_ref']
chk = get_optimal_chk(varr_ref, dtype=float)

Y_fm_chk = open_minian(int_dir)['Y_fm_chk']
Y_hw_chk = open_minian(int_dir)['Y_hw_chk']

initialize(Y_fm_chk, 
           Y_hw_chk, 
           param_save_minian, 
           param_seeds_init, 
           param_pnr_refine, 
           param_ks_refine, 
           param_seeds_merge, 
           param_initialize, 
           param_init_merge, 
           int_dir, 
           chk)

Processing directory: /Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope
Starting max projection
Initializing seeds
constructing chunks
computing max projections
calculating local maximum
Refining seeds with PNR
selecting seeds
computing peak-noise ratio


/Users/annateruel/minicondax86/envs/minian/lib/python3.8/site-packages/minian/initialization.py:340: RuntimeWarning: divide by zero encountered in floor_divide
  for _, seed_sub in seeds.groupby(np.arange(len(seeds)) // chk_size):


Refining seeds with KS test
selecting seeds
performing KS test


/Users/annateruel/minicondax86/envs/minian/lib/python3.8/site-packages/minian/initialization.py:509: RuntimeWarning: divide by zero encountered in floor_divide
  for _, seed_sub in seeds.groupby(np.arange(len(seeds)) // chk_size):


Merging seeds
Y_hw_chk shape: (3794, 542, 606)
max_proj shape: (542, 606)
seeds_final shape: (19, 5)
seeds_final:     height  width  seeds  mask_pnr  mask_ks
0      104    390    2.0      True     True
1      191    321    2.0      True     True
2      206    120    2.0      True     True
3      245    208    2.0      True     True
4      252    209    2.0      True     True
5      255    234    3.0      True     True
6      258    192    3.0      True     True
7      258    236    3.0      True     True
8      262    209    2.0      True     True
9      265    210    2.0      True     True
10     266    186    1.0      True     True
11     267    186    1.0      True     True
12     269    210    2.0      True     True
13     272    159    2.0      True     True
14     281    218    2.0      True     True
15     283    214    2.0      True     True
16     284    212    2.0      True     True
17     288    211    2.0      True     True
18     295    208    2.0      True     True
comput

AssertionError: A contains NaN values

In [3]:
int_dir =  '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-098/annateruel/social-discrimination-test/AD23-098/customEntValHere/2024_02_20/11_57_46/My_V4_Miniscope'
minian_ds = open_minian(int_dir)
A = minian_ds['A']
C = minian_ds['C']
A_init = minian_ds['A_init']
C_init = minian_ds['C_init']
Y_fm_chk = minian_ds['Y_fm_chk']
C_chk = minian_ds['C_chk']
Y_hw_chk = minian_ds['Y_hw_chk']


In [ ]:
A.shape, C.shape

In [ ]:
print("A data sample:", A.isel(unit_id=0).values)
print("C data sample:", C.isel(unit_id=0).values)

In [ ]:
print("Y_fm_chk data sample:", Y_fm_chk.isel(frame=0, height=0, width=0).values)
print("C_chk data sample:", C_chk.isel(frame=0, unit_id=0).values)

In [ ]:
from minian.cnmf import unit_merge
param_init_merge = {"thres_corr": 0.8} 
A, C = unit_merge(A_init, C_init, **param_init_merge)


In [ ]:
C_chk = save_minian(
        C.rename("C_chk"),
        int_dir,
        overwrite=True,
        chunks={"unit_id": -1, "frame": chk[0]["frame"]},
    )

In [5]:
print("Does Y_hw_chk contain NaN values?", np.isnan(Y_hw_chk.values).any())

Does Y_hw_chk contain NaN values? False


In [ ]:
from minian.cnmf import update_background
os.environ["MINIAN_INTERMEDIATE"] = int_dir
b, f = update_background(Y_fm_chk, A, C_chk)


In [ ]:
from holoviews.operation.datashader import regrid
from holoviews.operation.datashader import regrid, datashade
from holoviews.operation.datashader import regrid, datashade

output_size = 100
hv.output(size=int(output_size * 0.55))
im_opts = dict(
    frame_width=500,
    aspect=A.sizes["width"] / A.sizes["height"],
    cmap="Viridis",
    colorbar=True,
)
cr_opts = dict(frame_width=750, aspect=1.5 * A.sizes["width"] / A.sizes["height"])
(
    regrid(
        hv.Image(
            A.max("unit_id").rename("A").compute().astype(np.float32),
            kdims=["width", "height"],
        ).opts(**im_opts)
    ).relabel("Initial Spatial Footprints")
    + regrid(
        hv.Image(
            C.rename("C").compute().astype(np.float32), kdims=["frame", "unit_id"]
        ).opts(cmap="viridis", colorbar=True, **cr_opts)
    ).relabel("Initial Temporal Components")
).cols(2)

In [ ]:
root_dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/24-02-ST-anna/AD23-127'

param_seeds_init = {
    "wnd_size": 1000,
    "method": "rolling",
    "stp_size": 500,
    "max_wnd": 15,
    "diff_thres": 3}
param_pnr_refine = {"noise_freq": 0.06, "thres": 1}
param_ks_refine = {"sig": 0.05}
param_seeds_merge = {"thres_dist": 10, "thres_corr": 0.8, "noise_freq": 0.06}
param_initialize = {"thres_corr": 0.8, "wnd": 10, "noise_freq": 0.06}
param_init_merge = {"thres_corr": 0.8}

for dirpath, dirnames, filenames in os.walk(root_dir):
    if 'My_V4_Miniscope' in dirnames:
        int_dir = os.path.join(dirpath, 'My_V4_Miniscope')
        os.environ["MINIAN_INTERMEDIATE"] = int_dir

        param_save_minian = {
            "dpath": int_dir,
            "meta_dict": dict(session=-1, animal=-2),
            "overwrite": True}

        varr_ref = open_minian(int_dir)['varr_ref']
        chk = get_optimal_chk(varr_ref, dtype=float)

        Y_fm_chk = open_minian(int_dir)['Y_fm_chk']
        Y_hw_chk = open_minian(int_dir)['Y_hw_chk']

        initialize(Y_fm_chk, 
                   Y_hw_chk, 
                   param_save_minian, 
                   param_seeds_init, 
                   param_pnr_refine, 
                   param_ks_refine, 
                   param_seeds_merge, 
                   param_initialize, 
                   param_init_merge, 
                   int_dir, 
                   chk)